In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
plt.style.use("fivethirtyeight")

# Data wrangling

In [8]:
df = pd.read_csv("blackjack_data.csv")